In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib

import tensorflow as tf


print(tf.__version__)

train_data_dir = pathlib.Path("./data/train")
val_data_dir = pathlib.Path("./data/validation")
test_data_dir = pathlib.Path("./data/test")


2.3.0


In [26]:
train_image_count = len(list(data_dir.glob('*/*.jpg')))
test_image_count = len(list(data_dir.glob('*/*.jpg')))
val_image_count = len(list(data_dir.glob('*/*.jpg')))

train_list_ds = tf.data.Dataset.list_files(str(train_data_dir/'*/*'))
val_list_ds = tf.data.Dataset.list_files(str(val_data_dir/'*/*'))
test_list_ds = tf.data.Dataset.list_files(str(test_data_dir/'*/*'))

In [5]:
for f in train_list_ds.take(3):
  print(f.numpy())

for f in val_list_ds.take(3):
  print(f.numpy())

for f in test_list_ds.take(3):
  print(f.numpy())

b'data\\train\\wagonr\\0dbdhxdm3v8s2-IN.jpg'
b'data\\train\\swift\\ps2wmqoz5al13-IN.jpg'
b'data\\train\\wagonr\\3oabrt1p3t5y2-IN.jpg'
b'data\\validation\\swift\\ps4npx6dvpfq3-IN.jpg'
b'data\\validation\\wagonr\\n8w5oev5q0fm-IN.jpg'
b'data\\validation\\wagonr\\xlmtu654vbd41-IN.jpg'
b'data\\test\\swift\\xcdie1murdc62-IN.jpg'
b'data\\test\\wagonr\\9ml3kjnbqf831-IN.jpg'
b'data\\test\\wagonr\\hrpfkuej46tv2-IN.jpg'


In [19]:
#data augmentation function
IMG_WIDTH=100
IMG_HEIGHT=100
def decode_img(img):
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32) 
  img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT]) 
  img = tf.image.random_flip_left_right(img)
  img = tf.image.random_flip_up_down(img)
  img = tf.image.random_brightness(img, 0.3)
  return img

def get_label(path):
  part_list = tf.strings.split(path, os.path.sep)
  return part_list[-2]

def process_path(file_path):
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  label = get_label(file_path)
  return img, label

In [27]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_labeled_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
#train_labeled_ds = train_labeled_ds.batch(10).prefetch(buffer_size=AUTOTUNE)

test_labeled_ds = test_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
#train_labeled_ds = test_labeled_ds.batch(10).prefetch(buffer_size=AUTOTUNE)

val_labeled_ds = val_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)


In [32]:
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([
        keras.layers.Flatten(input_shape=(100, 100, 3)),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(1)
    ])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

tensorboard_callback = keras.callbacks.TensorBoard()

training_history = model.fit(
    train_labeled_ds, 
    validation_data=val_labeled_ds, 
    epochs=35,
    steps_per_epoch=80,
    callbacks=[tensorboard_callback])

print(f"Average test loss: {np.average(training_history.history['loss'])}")
val_loss, val_acc = model.evaluate(val_labeled_ds, verbose=2)
test_loss, test_acc = model.evaluate(test_labeled_ds, verbose=2)

print(f"\nTest accuracy: {test_acc}")
print(f"\nValidation accuracy: {val_acc}")


ValueError: in user code:

    C:\Users\luke\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\luke\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\luke\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\luke\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\luke\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\luke\.conda\envs\tf-gpu\lib\site-packages\tensorflow\py

In [70]:

train_ds = train_ds.cache()
train_ds = train_ds.shuffle(10000)
train_ds = train_ds.repeat(23)
train_ds = train_ds.batch(128)
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

In [71]:
for image, label in train_ds:
  print("Image shape: ", label)


Image shape:  tf.Tensor(
[b'wagonr' b'swift' b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift'
 b'swift' b'swift' b'swift' b'wagonr' b'wagonr' b'swift' b'wagonr'
 b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr' b'wagonr' b'swift'
 b'swift' b'swift' b'wagonr' b'wagonr' b'swift' b'swift' b'swift' b'swift'
 b'swift' b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift'
 b'swift' b'swift' b'swift' b'wagonr' b'wagonr' b'swift' b'wagonr'
 b'wagonr' b'swift' b'swift' b'swift' b'swift' b'wagonr' b'swift' b'swift'
 b'swift' b'swift' b'swift' b'wagonr' b'wagonr' b'swift' b'wagonr'
 b'swift' b'wagonr' b'wagonr' b'wagonr' b'swift' b'swift' b'wagonr'
 b'wagonr' b'swift' b'wagonr' b'swift' b'swift' b'wagonr' b'wagonr'
 b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift'
 b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift'
 b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr' b'wagonr'
 b'swift' b'swift' b'wagonr' b'swift' b'swift' b'swift' b'swift' b'swift'
 b'

Image shape:  tf.Tensor(
[b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr' b'wagonr'
 b'wagonr' b'wagonr' b'swift' b'wagonr' b'wagonr' b'wagonr' b'wagonr'
 b'wagonr' b'swift' b'wagonr' b'swift' b'swift' b'swift' b'wagonr'
 b'wagonr' b'swift' b'swift' b'wagonr' b'wagonr' b'swift' b'swift'
 b'wagonr' b'wagonr' b'swift' b'swift' b'swift' b'swift' b'swift'
 b'wagonr' b'swift' b'swift' b'swift' b'wagonr' b'wagonr' b'wagonr'
 b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift'
 b'swift' b'swift' b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr'
 b'wagonr' b'swift' b'swift' b'wagonr' b'wagonr' b'wagonr' b'swift'
 b'wagonr' b'swift' b'swift' b'swift' b'wagonr' b'wagonr' b'swift'
 b'wagonr' b'swift' b'wagonr' b'wagonr' b'swift' b'swift' b'swift'
 b'swift' b'wagonr' b'wagonr' b'swift' b'swift' b'swift' b'swift'
 b'wagonr' b'swift' b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift'
 b'wagonr' b'swift' b'wagonr' b'swift' b'swift' b'swift' b'wagonr'
 b'swift' b'swift' b'wagon

Image shape:  tf.Tensor(
[b'swift' b'swift' b'swift' b'wagonr' b'wagonr' b'swift' b'wagonr'
 b'swift' b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift'
 b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift' b'swift' b'wagonr'
 b'swift' b'swift' b'swift' b'wagonr' b'wagonr' b'swift' b'wagonr'
 b'wagonr' b'wagonr' b'swift' b'swift' b'swift' b'swift' b'wagonr'
 b'swift' b'wagonr' b'swift' b'wagonr' b'wagonr' b'swift' b'swift'
 b'swift' b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr' b'wagonr'
 b'swift' b'swift' b'swift' b'wagonr' b'swift' b'swift' b'wagonr'
 b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift'
 b'wagonr' b'wagonr' b'swift' b'swift' b'swift' b'swift' b'swift' b'swift'
 b'wagonr' b'swift' b'wagonr' b'wagonr' b'swift' b'swift' b'wagonr'
 b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr'
 b'swift' b'swift' b'wagonr' b'swift' b'wagonr' b'swift' b'swift' b'swift'
 b'swift' b'wagonr' b'swift' b'wagonr' b'wagonr' b'swift' b'wagonr'
 b'wagonr' b

Image shape:  tf.Tensor(
[b'swift' b'wagonr' b'swift' b'wagonr' b'wagonr' b'swift' b'swift'
 b'swift' b'wagonr' b'swift' b'swift' b'wagonr' b'swift' b'swift'
 b'wagonr' b'swift' b'wagonr' b'wagonr' b'swift' b'swift' b'swift'
 b'swift' b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr'
 b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift' b'swift' b'swift'
 b'wagonr' b'swift' b'wagonr' b'swift' b'swift' b'swift' b'swift'
 b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift' b'swift'
 b'swift' b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift' b'swift'
 b'swift' b'wagonr' b'swift' b'swift' b'swift' b'swift' b'wagonr'
 b'wagonr' b'swift' b'swift' b'swift' b'swift' b'swift' b'swift' b'wagonr'
 b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr'
 b'swift' b'wagonr' b'swift' b'swift' b'wagonr' b'wagonr' b'wagonr'
 b'swift' b'swift' b'swift' b'swift' b'swift' b'wagonr' b'swift' b'swift'
 b'swift' b'swift' b'swift' b'wagonr' b'wagonr' b'swift' b'wagonr'
 b'wagonr' b'wagon

Image shape:  tf.Tensor(
[b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr' b'wagonr' b'swift'
 b'swift' b'swift' b'swift' b'swift' b'wagonr' b'wagonr' b'swift'
 b'wagonr' b'swift' b'wagonr' b'swift' b'swift' b'wagonr' b'swift'
 b'swift' b'wagonr' b'wagonr' b'swift' b'wagonr' b'wagonr' b'swift'
 b'wagonr' b'wagonr' b'wagonr' b'swift' b'swift' b'swift' b'swift'
 b'swift' b'swift' b'wagonr' b'wagonr' b'swift' b'swift' b'swift'
 b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift'
 b'swift' b'swift' b'swift' b'swift' b'swift' b'swift' b'swift' b'swift'
 b'swift' b'swift' b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'wagonr'
 b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift' b'swift'
 b'swift' b'swift' b'swift' b'swift' b'wagonr' b'wagonr' b'swift' b'swift'
 b'swift' b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr' b'swift'
 b'wagonr' b'swift' b'swift' b'swift' b'wagonr' b'swift' b'swift'
 b'wagonr' b'swift' b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift'
 b'swift' b'wago

...

Image shape:  tf.Tensor(
[b'swift' b'swift' b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr'
 b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr' b'swift' b'swift'
 b'swift' b'swift' b'swift' b'wagonr' b'swift' b'swift' b'swift' b'wagonr'
 b'wagonr' b'wagonr' b'swift' b'wagonr' b'wagonr' b'wagonr' b'wagonr'
 b'wagonr' b'swift' b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift'
 b'wagonr' b'swift' b'wagonr' b'wagonr' b'swift' b'swift' b'wagonr'
 b'wagonr' b'swift' b'swift' b'swift' b'swift' b'swift' b'wagonr'
 b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr' b'wagonr' b'wagonr'
 b'swift' b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift' b'swift'
 b'swift' b'swift' b'swift' b'swift' b'wagonr' b'swift' b'wagonr' b'swift'
 b'swift' b'wagonr' b'wagonr' b'swift' b'swift' b'swift' b'wagonr'
 b'wagonr' b'swift' b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'wagonr'
 b'swift' b'swift' b'swift' b'swift' b'swift' b'swift' b'swift' b'wagonr'
 b'wagonr' b'swift' b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr'
 

Image shape:  tf.Tensor(
[b'swift' b'wagonr' b'swift' b'swift' b'swift' b'wagonr' b'wagonr'
 b'swift' b'wagonr' b'wagonr' b'wagonr' b'swift' b'swift' b'swift'
 b'swift' b'swift' b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift'
 b'wagonr' b'swift' b'swift' b'wagonr' b'swift' b'wagonr' b'wagonr'
 b'wagonr' b'swift' b'swift' b'wagonr' b'swift' b'wagonr' b'swift'
 b'swift' b'swift' b'swift' b'swift' b'wagonr' b'swift' b'wagonr' b'swift'
 b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift' b'swift' b'swift'
 b'wagonr' b'wagonr' b'wagonr' b'swift' b'swift' b'wagonr' b'wagonr'
 b'wagonr' b'swift' b'wagonr' b'swift' b'swift' b'wagonr' b'wagonr'
 b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr' b'wagonr' b'wagonr'
 b'swift' b'wagonr' b'swift' b'swift' b'wagonr' b'wagonr' b'swift'
 b'wagonr' b'swift' b'swift' b'wagonr' b'wagonr' b'swift' b'swift'
 b'wagonr' b'swift' b'swift' b'wagonr' b'wagonr' b'swift' b'swift'
 b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift'
 b'wagonr' b'wagonr' 

Image shape:  tf.Tensor(
[b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr' b'wagonr' b'swift'
 b'swift' b'swift' b'swift' b'wagonr' b'wagonr' b'wagonr' b'swift'
 b'swift' b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift'
 b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr'
 b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr' b'wagonr' b'wagonr'
 b'swift' b'swift' b'swift' b'wagonr' b'swift' b'swift' b'swift' b'wagonr'
 b'swift' b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr'
 b'wagonr' b'wagonr' b'swift' b'wagonr' b'wagonr' b'wagonr' b'swift'
 b'swift' b'swift' b'wagonr' b'wagonr' b'swift' b'swift' b'swift'
 b'wagonr' b'swift' b'swift' b'wagonr' b'wagonr' b'wagonr' b'wagonr'
 b'wagonr' b'wagonr' b'swift' b'swift' b'wagonr' b'swift' b'wagonr'
 b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift'
 b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift' b'swift'
 b'swift' b'swift' b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr'
 b'swift' b

Image shape:  tf.Tensor(
[b'swift' b'swift' b'wagonr' b'swift' b'swift' b'wagonr' b'swift'
 b'wagonr' b'wagonr' b'swift' b'swift' b'wagonr' b'wagonr' b'swift'
 b'swift' b'swift' b'swift' b'wagonr' b'swift' b'swift' b'wagonr' b'swift'
 b'swift' b'swift' b'wagonr' b'swift' b'swift' b'wagonr' b'wagonr'
 b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr' b'swift' b'swift'
 b'swift' b'swift' b'swift' b'swift' b'swift' b'wagonr' b'wagonr' b'swift'
 b'swift' b'swift' b'wagonr' b'swift' b'swift' b'swift' b'wagonr' b'swift'
 b'wagonr' b'swift' b'wagonr' b'swift' b'swift' b'swift' b'swift'
 b'wagonr' b'swift' b'swift' b'swift' b'swift' b'wagonr' b'swift'
 b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift' b'swift' b'wagonr'
 b'swift' b'wagonr' b'swift' b'swift' b'swift' b'swift' b'swift' b'swift'
 b'swift' b'swift' b'swift' b'swift' b'swift' b'swift' b'wagonr' b'swift'
 b'swift' b'wagonr' b'swift' b'wagonr' b'wagonr' b'wagonr' b'swift'
 b'swift' b'swift' b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr

Image shape:  tf.Tensor(
[b'swift' b'swift' b'swift' b'swift' b'wagonr' b'swift' b'wagonr' b'swift'
 b'swift' b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr'
 b'swift' b'swift' b'swift' b'wagonr' b'swift' b'wagonr' b'swift'
 b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift' b'swift' b'wagonr'
 b'wagonr' b'swift' b'swift' b'swift' b'swift' b'swift' b'wagonr' b'swift'
 b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr'
 b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift'
 b'swift' b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr' b'wagonr'
 b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr' b'swift'
 b'swift' b'wagonr' b'swift' b'swift' b'swift' b'swift' b'swift' b'swift'
 b'wagonr' b'swift' b'wagonr' b'wagonr' b'wagonr' b'swift' b'swift'
 b'wagonr' b'swift' b'swift' b'wagonr' b'swift' b'swift' b'wagonr'
 b'wagonr' b'swift' b'swift' b'wagonr' b'swift' b'wagonr' b'swift'
 b'wagonr' b'swift' b'swift' b'swift' b'swift' b'swift' b'wagonr' b's

In [78]:
for image,label in ds:  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
  print(image.shape, label)

(128, 224, 224, 3) tf.Tensor(
[b'wagonr' b'wagonr' b'wagonr' b'wagonr' b'swift' b'swift' b'swift'
 b'swift' b'swift' b'wagonr' b'swift' b'wagonr' b'wagonr' b'swift'
 b'wagonr' b'swift' b'wagonr' b'swift' b'swift' b'swift' b'swift'
 b'wagonr' b'swift' b'wagonr' b'wagonr' b'swift' b'wagonr' b'wagonr'
 b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr'
 b'swift' b'swift' b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr'
 b'swift' b'swift' b'swift' b'swift' b'swift' b'swift' b'wagonr' b'wagonr'
 b'wagonr' b'wagonr' b'wagonr' b'swift' b'swift' b'swift' b'swift'
 b'swift' b'wagonr' b'swift' b'swift' b'wagonr' b'wagonr' b'wagonr'
 b'swift' b'swift' b'wagonr' b'swift' b'wagonr' b'swift' b'swift'
 b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr' b'wagonr' b'swift'
 b'swift' b'swift' b'wagonr' b'wagonr' b'swift' b'wagonr' b'wagonr'
 b'wagonr' b'wagonr' b'wagonr' b'swift' b'wagonr' b'wagonr' b'wagonr'
 b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr' b'swift' b'wagonr'
 b'swift' b'wa

In [81]:
# Classifiers
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
# Metrics
from sklearn.metrics import precision_recall_fscore_support

In [35]:
import tensorflow as tf
import datetime

#!rm -rf ./logs/

%load_ext tensorboard


 

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [37]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(train_list_ds, 
          validation_data=val_list_ds, 
          epochs=5, 
          callbacks=[tensorboard_callback])

Epoch 1/5


ValueError: in user code:

    C:\Users\luke\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\luke\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\luke\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\luke\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\luke\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\luke\.conda\envs\tf-gpu\lib\site-packages\tensorflow\py

In [5]:
%tensorboard --logdir logs/fit


Reusing TensorBoard on port 6006 (pid 164724), started 0:04:16 ago. (Use '!kill 164724' to kill it.)

In [ ]:
!kill 164724